In [1]:
# env setup
import os
import sys

os.chdir("/home/yxjiang/source/ml_playground")

print(os.getcwd())

/home/yxjiang/source/ml_playground


In [3]:
# data downloading
from util import data_util
import pandas as pd
from collections import defaultdict

# dataset_url="https://s3.amazonaws.com/fast-ai-nlp/dbpedia_csv.tgz"
dataset_url="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dest_dir = "/tmp/data"
dataset_folder_path = os.path.join(dest_dir, "aclImdb")
data_util.download_data(url=dataset_url, dest_dir=dest_dir)

# generate word to id mapping
word_to_id, word_list = data_util.get_vocabulary(folder_path=dataset_folder_path, file_suffix="vocab")
print("There size of vocabulary is :", len(word_to_id))

# generate class id to name mapping
# class_to_name = defaultdict(str)
# with open(os.path.join(dataset_folder_path, "classes.txt"), "r") as f:
#     for i, class_name in enumerate(f):
#         class_to_name[i] = class_name.strip()
# print("There class mapping:", class_to_name.items())

Destination folder [/tmp/data] exists.
Target file [aclImdb_v1.tar.gz] exists, skip downloading.
Start to extract [/tmp/data/aclImdb_v1.tar.gz] to [/tmp/data]...
File extracted
Processing vocabulary from [/tmp/data/aclImdb].
There size of vocabulary is : 89527


In [4]:
# transform, dataset and dataloader
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


class CutOrPadTransform:
    """
    Shape all sentences to the equal length.
    """
    def __init__(self, config):
        self.config = config
    
    def __call__(self, input):
        if len(input["words"]) >= config.sentence_max_length:
            input["words"] = input["words"][:config.sentence_max_length]
        else:
            input["words"].extend([" "] * (config.sentence_max_length - len(input["words"])))
        return input


class WordsToIdsTransform:
    """
    Convert the list of words to embeddings.
    """
    def __init__(self, config, word_to_id):
        self.config = config
        self.word_to_id = word_to_id
    
    def __call__(self, input):
        input["word_ids"] = torch.tensor([self.word_to_id[w.lower()] for w in input["words"]], dtype=torch.long)
        # del input['words']
        return input


class MovieReviewDataset(Dataset):
    def __init__(self, config, pos_data_folder, neg_data_folder, word_to_id, transform):
        self.config = config
        self.word_to_id = word_to_id
        self.data = []
        # read all data into memory
        for filename in os.listdir(pos_data_folder):
            if filename.endswith(".txt"):
                with open(os.path.join(pos_data_folder, filename), "r") as f:
                    self.data.append((f.readline(), 1))

        for filename in os.listdir(neg_data_folder):
            if filename.endswith(".txt"):
                with open(os.path.join(neg_data_folder, filename), "r") as f:
                    self.data.append((f.readline(), 0))

        self.transform = transform
    
    def __getitem__(self, idx):
        words = [w.strip() for w in self.data[idx][0].strip().split(" ")]
        label = self.data[idx][1]
        input = self.transform({"words": words, "label": label})
        # print(input["words"], "\n", input["word_ids"], "\n", input["label"])
        return input["words"], input["word_ids"], input["label"]
        

    def __len__(self):
        return len(self.data)

In [53]:
# model
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, config, vocabulary_size):
        super().__init__()
        self.config = config
        self.embed = nn.Embedding(vocabulary_size, config.word_embedding_length)
        self.hidden_size = 128
        self.num_layers = 1
        self.directions = 1
        self.rnn = nn.RNN(input_size=config.word_embedding_length, hidden_size=self.hidden_size, num_layers=self.num_layers)
        self.fc1 = nn.Linear(in_features=self.hidden_size, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=config.num_classes)

    def forward(self, x):
        batch = x.shape[0]
        x = self.embed(x)  # (batch, sentence_length, embedding_dim)
        x = x.permute(1, 0, 2).contiguous()  # (sentence_length, batch, embedding_dim)

        h0 = torch.zeros((self.num_layers * self.directions, batch, self.hidden_size)).to(device)
        output, ht = self.rnn(x, h0)
        ht = ht.permute(1, 0, 2)  # (batch, num_layer * directions, embedding_dim)
        ht = ht.contiguous().view(batch, self.num_layers * self.directions, self.hidden_size)
        x = F.relu(self.fc1(ht))
        x = F.relu(self.fc2(x))
        x = x.view(batch, -1)
        return x



class TextCNN(nn.Module):
    def __init__(self, config, vocabulary_size, conv_layer_sizes):
        super().__init__()
        self.config = config
        self.embed = nn.Embedding(vocabulary_size, config.word_embedding_length)

        self.convs = []
        self.max_over_time_pooling = []
        for size in conv_layer_sizes:
            self.convs.append(nn.Conv2d(1, 1, kernel_size=(size, config.word_embedding_length)).to(device))
            self.max_over_time_pooling.append(nn.MaxPool2d((config.sentence_max_length - size + 1, 1)).to(device))

        self.dropout = nn.Dropout(config.dropout)
        self.fc = nn.Linear(len(self.convs), config.num_classes)


    def forward(self, x):
        batch = x.shape[0]
        x = torch.unsqueeze(self.embed(x), 1)  # [NCHW], add channel to dimension 1
        # convs
        xs = []
        for i in range(len(self.convs)):
            xs.append(self.config.activation(self.convs[i](x)))  # conv modules
            xs[i] = self.max_over_time_pooling[i](xs[i])  # max over time pooling modules

        x = torch.cat(xs, dim=-1)
        x = self.dropout(x)
        x = self.fc(x)
        x = x.view(batch, -1)
        
        return x

In [54]:
# trainer
from torch.utils.tensorboard import SummaryWriter

def train(model, config, train_dataloader, test_dataloader, device, check_interval=5000):
    criteria = config.criteria()
    optimizer = config.optimizer(model.parameters(), config.lr)
    start = time.time()
    counts = 0
    writer = SummaryWriter()
    # writer.add_graph(model)
    torch.backends.cudnn.benchmark = True
    for epoch in range(config.epochs):
        for i, (words, word_ids, labels) in enumerate(train_dataloader):
            counts += labels.shape[0]
            optimizer.zero_grad()
            output = model(word_ids.to(device))
            loss = criteria(output, labels.to(device))
            loss.backward()
            optimizer.step()
            if ((epoch + 1) * i) % check_interval == 0:
                print("[%d seconds](epoch: %d/%d)[%d samples] loss: %.3f." % (time.time() - start, epoch + 1, config.epochs, counts, loss.mean().item()))
                # eval on test dataset
                model.eval()
                with torch.no_grad():
                    acc_eval_loss = 0.0
                    batches = 0
                    for j, (words, eval_word_ids, eval_label) in enumerate(test_dataloader):
                        eval_output = model(eval_word_ids.to(device))
                        eval_loss = criteria(eval_output, eval_label.to(device))
                        acc_eval_loss += eval_loss.item()
                        batches += 1
                    print('eval loss: %.3f' % (acc_eval_loss / batches))
                model.train()
                writer.add_scalar('Loss/train', loss.mean().item(), (epoch + 1) * i)

In [58]:
# configs
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

class Config:
    def __init__(self):
        self.num_classes = 2
        self.sentence_max_length = 30
        self.word_embedding_length = 32
        self.activation = F.relu
        self.criteria = nn.CrossEntropyLoss
        self.optimizer = optim.SGD
        self.lr = 0.0005
        self.epochs = 100
        self.batch_size = 1024
        self.dropout = 0.5

In [59]:
# put everything together
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = Config()

pos_train_data_folder = os.path.join(dataset_folder_path, "train/pos")
neg_train_data_folder = os.path.join(dataset_folder_path, "train/neg")
train_dataset = MovieReviewDataset(config, pos_train_data_folder, neg_train_data_folder, word_to_id, 
                            transform=transforms.Compose([
                                CutOrPadTransform(config), WordsToIdsTransform(config, word_to_id)
                            ]))

pos_test_data_folder = os.path.join(dataset_folder_path, "test/pos")
neg_test_data_folder = os.path.join(dataset_folder_path, "test/neg")
test_dataset = MovieReviewDataset(config, pos_test_data_folder, neg_test_data_folder, word_to_id, 
                            transform=transforms.Compose([
                                CutOrPadTransform(config), WordsToIdsTransform(config, word_to_id)
                            ]))

train_dataloader = DataLoader(dataset=train_dataset, batch_size=config.batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=config.batch_size)

model = TextCNN(config, len(word_to_id), conv_layer_sizes=(3, 4, 5, 6, 7)).to(device)
# model = RNN(config, len(word_list)).to(device)

train(model, config, train_dataloader, test_dataloader, device)

[0 seconds](epoch: 1/100)[1024 samples] loss: 0.815.
eval loss: 0.749
[2 seconds](epoch: 2/100)[26024 samples] loss: 0.783.
eval loss: 0.741
[5 seconds](epoch: 3/100)[51024 samples] loss: 0.812.
eval loss: 0.735
[7 seconds](epoch: 4/100)[76024 samples] loss: 0.790.
eval loss: 0.729
[10 seconds](epoch: 5/100)[101024 samples] loss: 0.782.
eval loss: 0.724
[12 seconds](epoch: 6/100)[126024 samples] loss: 0.784.
eval loss: 0.720
[15 seconds](epoch: 7/100)[151024 samples] loss: 0.760.
eval loss: 0.716
[17 seconds](epoch: 8/100)[176024 samples] loss: 0.762.
eval loss: 0.713
[20 seconds](epoch: 9/100)[201024 samples] loss: 0.764.
eval loss: 0.710
[22 seconds](epoch: 10/100)[226024 samples] loss: 0.754.
eval loss: 0.708
[25 seconds](epoch: 11/100)[251024 samples] loss: 0.768.
eval loss: 0.706
[27 seconds](epoch: 12/100)[276024 samples] loss: 0.743.
eval loss: 0.704
[30 seconds](epoch: 13/100)[301024 samples] loss: 0.714.
eval loss: 0.703
[33 seconds](epoch: 14/100)[326024 samples] loss: 0.752.

In [ ]:
import torch
import torch.nn as nn

sentence_length = 5
batch_size = 3
input_dim = 6
num_direction = 1
layer = 1
hidden_size = 7

rnn = nn.RNN(input_dim, hidden_size, layer)
x = torch.randn(sentence_length, batch_size, input_dim)
h0 = torch.randn(num_direction * layer, batch_size, hidden_size)

output, hn = rnn(x, h0)
print(output.shape, hn.shape)

print(hn)
print(output)